In [ ]:
%matplotlib inline

import os
import json
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

from IPython.display import Image, display

px = 1 / plt.rcParams['figure.dpi']

sns.set_theme(
    style='darkgrid',
    font_scale=2,
    rc={
        'figure.figsize': (1920 * px, 1080 * px),
        'axes.labelsize': 48,
        'xtick.labelsize': 36,
        'ytick.labelsize': 36,
        'legend.fontsize': 36,
        'grid.linewidth': 2,
    }
)

datecode = os.path.basename(os.getcwd())
print(f"Experiment code: {datecode}")

configuration = json.load(open(f'configuration_{datecode}.txt'))

def get_dataframe():
    return pd.read_csv(f'data_{datecode}.csv', index_col=0)

# Evaluation

## Convergence speeds over the training simulations

In [ ]:
df = get_dataframe()

window_size = 100
agents = ['MultiFedRL', 'FlexiFed', 'ClusterFed', 'Independent']

df = df[(df['Train'] == True) & (df['Agent'] != 'Random')]

df = df.groupby(['Agent', 'Experiment', 'Environment', 'Iteration']).rolling(window=window_size, min_periods=1, on='Step').mean()

plot = sns.lineplot(
    data=df, x='Step', y='Rewards', hue='Agent', style='Agent', style_order=agents, hue_order=agents,
    markers=True, markersize=24, linewidth=3, markevery=100,
)

plot.legend(facecolor='white', loc='lower right', markerscale=3)

In [ ]:
df = get_dataframe()

window_size = 100
threshold = 0.9
agents = ['Independent', 'ClusterFed', 'FlexiFed', 'MultiFedRL']

num_experiments = df['Experiment'].max() + 1
num_environments = configuration['num_environments']

# Get the final performance
test_df = df[(df['Train'] == False)]
test_df = test_df.groupby(['Agent', 'Experiment','Iteration']).mean()
test_df = test_df.groupby(['Experiment']).max()

# Get the convergence performance
train_df = df[(df['Train'] == True)]
train_df = train_df.groupby(['Agent', 'Experiment', 'Environment', 'Iteration'], as_index=False).rolling(window=window_size, on='Step').mean()
train_df = train_df.groupby(['Agent', 'Experiment', 'Iteration', 'Step']).mean()

# Get the requests
requests_df = df[(df['Train'] == True)].groupby(['Agent', 'Experiment', 'Iteration', 'Step']).sum()

performance = []

print(f"\\begin{{tabular}}{{c|{'c' * (num_experiments)}}}")
print(f"        \\hline\\hline")
print(f"        \\multirow{{2}}{{*}}{{\\textbf{{Method}}}} & \\multicolumn{{{num_experiments}}}{{c}}{{\\textbf{{Environment set}}}} \\\\")
print('        & ' + ' & '.join([f"Set {i}" for i in range(num_experiments)]) + ' \\\\')

for agent in agents:
    convergence = []
    for exp in range(num_experiments):    
        curve = train_df.loc[agent].loc[exp].loc[0]
        exceed = curve[curve['Rewards'] > test_df.loc[exp]['Rewards'] * threshold]
        if len(exceed) > 0:
            index = exceed.index[0]
            convergence.append(requests_df.loc[agent].loc[exp].loc[0].loc[index]['Requests'])
        else:
            convergence.append(np.inf)
    performance.append(convergence)
    
best = np.argmin(performance, axis=0)

for a, agent in enumerate(agents):    
    string = '& ' + ' & '.join([f"\\textbf{{{c}}}" if best[i] == a else f"{c}" for i, c in enumerate(performance[a])]) + ' \\\\'
    print(f"        \\hline")
    print(f"        \\textit{{{agent.replace('Agent', '')}}} {string.replace('inf', '-')}")
    
print(f"        \\hline\\hline")
print(f"    \\end{{tabular}}")

## Collected rewards over the testing simulations

In [ ]:
df = get_dataframe()
df = df[df['Train'] == False]

agents = ['Independent', 'ClusterFed', 'FlexiFed', 'MultiFedRL']
num_experiments = df['Experiment'].max() + 1

print(f"\\begin{{tabular}}{{c|{'c' * (num_experiments + 1)}}}")
print(f"        \\hline\\hline")
print(f"        \\multirow{{2}}{{*}}{{\\textbf{{Method}}}} & \\multicolumn{{{num_experiments+1}}}{{c}}{{\\textbf{{Environment set}}}} \\\\")
print('        & ' + ' & '.join([f"Set {i}" for i in range(num_experiments)]) + ' & Average \\\\')

performance = []

df = df.groupby(['Agent', 'Experiment', 'Iteration']).mean()

for agent in agents:
    reward_mean = [df.loc[agent].loc[exp]['Rewards'].iloc[0] for exp in range(num_experiments)]
    reward_mean.append(sum(reward_mean) / len(reward_mean))
    performance.append(reward_mean)

best = np.argmax(performance, axis=0)

for a, agent in enumerate(agents):
    string = '& ' + ' & '.join([f"\\textbf{{{m:.3f}}}" if best[i] == a else f"{m:.3f}" for i, m in enumerate(performance[a])]) + ' \\\\'
    print(f"        \\hline")
    print(f"        \\textit{{{agent.replace('Agent', '')}}} {string}")
print(f"        \\hline\\hline")
print(f"    \\end{{tabular}}")

In [ ]:
df = get_dataframe()
plot = sns.boxplot(
    data=df, x='Experiment', y='Reward', hue='Agent',
    showfliers=False,
)

# Statistics

## Users

In [ ]:
df = get_dataframe()

df = df[df['Agent'] == 'MultiFed']

print("\n---Mean---")
print(df.mean(numeric_only=True))

print("\n---Max---")
print(df.max(numeric_only=True))

print("\n---Min---")
print(df.min(numeric_only=True))

print("\n---Std---")
print(df.std(numeric_only=True))

# Details

## Settings

In [ ]:
print(json.dumps(configuration, indent=4))

## Environments

In [ ]:
for exp in [exp for exp in sorted(os.listdir(f'.')) if 'Exp' in exp]:
    for env in [env for env in sorted(os.listdir(f'./{exp}')) if 'Env' in env]:
        print(f'{exp} {env}')
        display(Image(filename=f'./{exp}/{env}/{env}.png'))

## Model

In [ ]:
for image in [image for image in sorted(os.listdir('.')) if '.png' in image]:
    print(image)
    display(Image(filename=f'{image}'))

## Log

In [ ]:
for l in open(f'log_{datecode}.txt'):
    print(l.strip())